# Advanced RNN - 3
- CuDNNGRU & CuDNNLSTM implementation
- Note that you need to install Tensorflow > 1.4 & Keras > 2.08 to implement 
- This source code is running on i5-7500 & GTX 1060 6GB

In [1]:
from keras.datasets import imdb
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

### Import dataset
- IMDB dataset in Keras datasets
- doc: https://keras.io/datasets/

In [2]:
num_words = 30000
maxlen = 300

In [3]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [5]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, maxlen = maxlen, padding = 'post')
X_test = pad_sequences(X_test, maxlen = maxlen, padding = 'post')

In [6]:
X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000, 300, 1)
(25000, 300, 1)
(25000,)
(25000,)


### LSTM
- Naive LSTM model without CuDNN

In [8]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape = (300,1), return_sequences = True))
    model.add(LSTM(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [9]:
model = lstm_model()

In [10]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

CPU times: user 1min 20s, sys: 3.25 s, total: 1min 23s
Wall time: 1min 15s


In [11]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 51.08%


### GRU
- Naive GRU model without CuDNN

In [12]:
def gru_model():
    model = Sequential()
    model.add(GRU(50, input_shape = (300,1), return_sequences = True))
    model.add(GRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [13]:
model = gru_model()

In [14]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

CPU times: user 1min 26s, sys: 2.43 s, total: 1min 28s
Wall time: 1min 20s


In [15]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 52.01%


### CuDNN LSTM

In [16]:
def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(50, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNLSTM(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [17]:
model = cudnn_lstm_model()

In [18]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

CPU times: user 1min 15s, sys: 1.66 s, total: 1min 16s
Wall time: 1min 12s


In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 52.24%


### CuDNN GRU

In [20]:
def cudnn_gru_model():
    model = Sequential()
    model.add(CuDNNGRU(50, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNGRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [21]:
model = cudnn_gru_model()

In [22]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

CPU times: user 1min 21s, sys: 1.46 s, total: 1min 23s
Wall time: 1min 19s


In [23]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 51.84%
